In [205]:
import pandas
import math
import re

########################################################
evidence_file = 'data/evidence.txt'
filtered_evidence_file = 'data/evidence_filtered.txt'
protein_groups_file = 'data/proteinGroups.txt'
phospho_peptide_file = 'data/Phospho (STY)Sites.txt'
our_group_name = 'PYND'
other_group_names = ['APUBSCRAWL', 'ONION', 'ET0H', 'SHMOO', 'WHANGEE'] # Case doesn't matter
########################################################

In [27]:
def filter_tsv(input_file, output_file, col_name, pattern):
    """
    For a tab-delimited file input_file with the first line as a header,
    writes (to output_file) every line whose entry at column 'col_name' matches
    the regex pattern 'pattern'.
    """
    if isinstance(pattern, str):
        pattern = re.compile(pattern)
        
    with open(output_file, 'w') as to_write:
        with open(input_file, 'r') as fp:
            index=-1
            n = 0
            for line in fp:
                parts = line.split('\t')
                if n == 0:
                    index = parts.index(col_name)
                    if index < 0:
                        raise ValueError(col_name + ' not found in ' + str(file))
                    to_write.write(line)
                else:
                    if pattern.match(parts[index]):
                        to_write.write(line)
                n += 1

# Test
# TODO delete output file
filter_tsv('test.tsv', 'output.tsv', 'evidence', re.compile('^keep'))
with open('output.tsv', 'r') as fp:
    assert ['a\tb\tevidence\tc\n', 'no\tno\tkeep\tno\n', 'no\tno\tkeeper\tno\n'] == fp.readlines()
import os
os.remove('output.tsv')
print('Tests passed')

Tests passed


In [28]:
# Now apply the filter to evidence.txt to get evidence_filtered.tsv
# This should keep only the lines for our group
filter_tsv(evidence_file, filtered_evidence_file, 'Experiment', '^[Pp][Yy][Nn][Dd]')
print('Done.')

Done.


In [124]:

class SiteString:
    """
    A peptide sequence with one or more amino acids mapped to a score or name
    """
    
    orig_seq = None
    sequence = None
    sites = None # Make a dict later
    
    def __init__(self, string):
        """
        string: A string from the data file like 'AFVNHM(8.97)M(-8.97)SSHSNHPGKR'
        """
        self.orig_seq = string
        self.sites = {}
        #pattern = '\\((-?[\\d.]+)\\)'
        pattern = '\\(([^\\)]+)\\)'
        offset = 0
        for match in re.finditer(pattern, string):
            to_value = match.group(1)
            try:
                to_value = float(to_value)
            except: pass
            self.sites[match.start() + offset] = to_value
            offset -= len(match.group(0))
        self.sequence = re.sub(pattern, '', string)
        
    def __str__(self):
        return self.orig_seq
    
    def __repr__(self):
        return self.orig_seq


# Test scored site parser
import pytest
test_sites = SiteString('AFVNHM(8.97)MM(-8.97)SSHSNH(1.0)PGKR')
assert 'AFVNHMMMSSHSNHPGKR' == test_sites.sequence
assert {6: 8.97, 8: -8.97, 14: 1.0} == test_sites.sites

test_sites_2 = SiteString('ABCD(efg)HIJK')
assert 'ABCDHIJK' == test_sites_2.sequence
assert {4: 'efg'} == test_sites_2.sites
assert 'ABCD(efg)HIJK' == str(test_sites_2)

test_sites_empty = SiteString('')
assert '' == test_sites_empty.sequence
assert {} == test_sites_empty.sites
assert '' == str(test_sites_empty.sequence)

print("Tests passed")

Tests passed


In [148]:

def parse_evidence(file, cols_to_drop=['Raw file', 'MS/MS m/z', 'Charge', 'm/z', 'Mass', 'Resolution', 'Uncalibrated - Calibrated m/z [ppm]', 'Uncalibrated - Calibrated m/z [Da]', 'Mass Error [ppm]', 'Mass Error [Da]', 'Uncalibrated Mass Error [ppm]', 'Uncalibrated Mass Error [Da]', 'Max intensity m/z 0', 'Retention time', 'Retention length', 'Calibrated retention time', 'Calibrated retention time start', 'Calibrated retention time finish', 'Retention time calibration', 'Match time difference', 'Match m/z difference', 'Match q-value']):
    
    df = pandas.read_table(file, header=0, index_col=0)

    # These are lists
    for m in [
            'Modifications',
            'Acetyl (Protein N-term)',
            'Proteins',
            'Leading proteins'
    ]:
        df[m] = df[m].map(lambda s: [st.strip() for st in str(s).split(';')])
        
    # And these are lists of ints
    for m in [
            'Oxidation (M) site IDs',
            'Phospho (STY) site IDs',
            'MS/MS IDs'
    ]:
        df[m] = df[m].map(lambda s: [int(st) for st in str(s).split(';')] if str(s) != 'nan' else [])
        
    # These have syntax like 'AFVNHM(8.97)M(-8.97)SSHSNHPGKR'
    for m in [
            'Oxidation (M) Score Diffs',
            'Phospho (STY) Score Diffs',
            'Phospho (STY) Probabilities',
            'Oxidation (M) Probabilities',
            'Modified sequence'
    ]:
        df[m] = df[m].map(lambda s: SiteString(str(s)) if str(s) != 'nan' else SiteString(''))
        
    # Make SiteString, but get rid of _s on ends
    df['Modified sequence'] = df['Modified sequence'].map(lambda s: SiteString(str(s)[1:-1]))
    
    # Some contain "Infinity", which float() understands but Pandas doesn't
    df['PEP'] = df['PEP'].map(lambda s: float(s))
    
    # Just use an empty list, not a list containing "Unmodified"
    df['Modifications'] = df['Modifications'].map(lambda lst: [] if lst == ['Unmodified'] else lst)

    df.drop(cols_to_drop, axis=1, inplace=True)
    
    return df
    

In [149]:
evidence = parse_evidence(filtered_evidence_file)
print(evidence.dtypes)
evidence

Length                           int64
Modifications                   object
Modified sequence               object
Oxidation (M) Probabilities     object
Phospho (STY) Probabilities     object
Oxidation (M) Score Diffs       object
Phospho (STY) Score Diffs       object
Acetyl (Protein N-term)         object
Oxidation (M)                    int64
Phospho (STY)                    int64
Missed cleavages                 int64
Proteins                        object
Leading proteins                object
Leading razor protein           object
Type                            object
Experiment                      object
Match score                    float64
Number of data points          float64
Number of scans                float64
Number of isotopic peaks       float64
PIF                            float64
Fraction of total spectrum     float64
Base peak fraction             float64
PEP                            float64
MS/MS Count                      int64
MS/MS Scan Number        

,Length,Modifications,Modified sequence,Oxidation (M) Probabilities,Phospho (STY) Probabilities,Oxidation (M) Score Diffs,Phospho (STY) Score Diffs,Acetyl (Protein N-term),Oxidation (M),Phospho (STY),...,Potential contaminant,id,Protein group IDs,Peptide ID,Mod. peptide ID,MS/MS IDs,Best MS/MS,AIF MS/MS IDs,Oxidation (M) site IDs,Phospho (STY) site IDs
Sequence,,,,,,,,,,,,,,,,,,,,,
AAAAEKNVPLYK,12,[],AAAAEKNVPLYK,,,,,[0],0,0,...,NaN,1,1258,0,0,[],NaN,NaN,[],[]
AAAAEKNVPLYQHLADLSK,19,[],AAAAEKNVPLYQHLADLSK,,,,,[0],0,0,...,NaN,7,1393,1,1,[3],3,NaN,[],[]
AAAAEKNVPLYQHLADLSK,19,[],AAAAEKNVPLYQHLADLSK,,,,,[0],0,0,...,NaN,19,1393,1,1,[],NaN,NaN,[],[]
AAAAGAGGAGDSGDAVTK,18,[],AAAAGAGGAGDSGDAVTK,,,,,[0],0,0,...,NaN,38,1889,2,2,[],NaN,NaN,[],[]
AAAALAGGK,9,[],AAAALAGGK,,,,,[0],0,0,...,NaN,43,1143,3,3,[23],23,NaN,[],[]
AAAALAGGK,9,[],AAAALAGGK,,,,,[0],0,0,...,NaN,60,1143,3,3,[],NaN,NaN,[],[]
AAAALAGGKK,10,[],AAAALAGGKK,,,,,[0],0,0,...,NaN,71,1143,4,4,[39],39,NaN,[],[]
AAAALAGGKK,10,[],AAAALAGGKK,,,,,[0],0,0,...,NaN,72,1143,4,4,"[40, 41]",41,NaN,[],[]
AAAALAGGKK,10,[],AAAALAGGKK,,,,,[0],0,0,...,NaN,98,1143,4,4,[],NaN,NaN,[],[]


In [152]:
def parse_protein_groups(f):
    #col_indicies = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 18, 19, 20, 21, 38, 39, 40, 41, 42, 43, 44, 45, 68, 69, 70, 71, 88, 89, 90, 91, 92, 93, 94, 95, 118, 119, 120, 121, 138, 139, 140, 141, 142, 143, 144, 145, 160, 161, 162, 163, 164, 165, 166, 167, 176, 177, 178, 179, 196, 197, 198, 199, 200, 201, 202, 203, 226, 227, 228, 229, 246, 247, 248, 249, 250, 251, 252, 253, 268, 277, 278, 279, 280, 297, 298, 299, 300, 301, 302, 303, 304, 327, 328, 329, 330, 347, 348, 349, 350, 351, 352, 353, 354, 377, 378, 379, 380, 397, 398, 399, 400, 401, 402, 403, 404, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433]
    df = pandas.read_table(f, header=0) # , usecols=col_indicies
    
    # These are lists
    for m in [
            'Protein IDs',
            'Majority protein IDs',
            'Fasta headers',
            'Peptide is razor',
        ]:
            df[m] = df[m].map(lambda s: [st.strip() for st in str(s).split(';')])
            
    # And these are lists of ints
    for m in [
            'Peptide IDs',
            'Peptide counts (all)',
            'Peptide counts (razor+unique)',
            'Mod. peptide IDs',
            'Evidence IDs',
            'MS/MS IDs',
            'Best MS/MS',
            'Oxidation (M) site IDs',
            'Phospho (STY) site IDs',
            'Oxidation (M) site positions',
            'Phospho (STY) site positions'
    ]:
        df[m] = df[m].map(lambda s: [int(st) for st in str(s).split(';')] if str(s) != 'nan' else [])
        
    # TODO: fix extra list columns
    print(df.dtypes)
    return df

In [200]:
import copy
def remove_cols(df, col_names_to_exclude):
    """
    Removes from the DataFrame every column whose name is a superset of any name in col_names_to_exclude
    Especially useful for filtering out the samples of other groups.
    """
    cols_copy = copy.copy(df.columns)
    for col in cols_copy:
        for group_name in col_names_to_exclude:
            if group_name.lower() in col.lower():
                df.drop(col, axis=1, inplace=True)
                break
    return df

def filter_other_groups(df):
    return remove_cols(df, other_group_names)

# TODO Test
df = pandas.DataFrame({'kerri' : pandas.Series([1., 2., 3., 4.]),
                       'ate' : pandas.Series([1., 2., 3., 4.]),
                       'our' : pandas.Series([1., 2., 3., 4.]),
                       'beloved' : pandas.Series([1., 2., 3., 4.]),
                       'pet' : pandas.Series([1., 2., 3., 4.]),
                       'chicken' : pandas.Series([1., 2., 3., 4.])})
df = remove_cols(df, ['love', 'ken'])
assert 4 == len(df.columns)
assert set(['kerri', 'ate', 'our', 'pet']) == set(df.columns)
print(df)
print("Tests passed")

   ate  kerri  our  pet
0    1      1    1    1
1    2      2    2    2
2    3      3    3    3
3    4      4    4    4
Tests passed


In [198]:
protein_groups = parse_protein_groups(protein_groups_file)
protein_groups = remove_cols(protein_groups, other_group_names)
print(protein_groups.dtypes)
protein_groups

Protein IDs                              object
Majority protein IDs                     object
Peptide counts (all)                     object
Peptide counts (razor+unique)            object
Peptide counts (unique)                  object
Fasta headers                            object
Number of proteins                        int64
Peptides                                  int64
Razor + unique peptides                   int64
Unique peptides                           int64
Peptides Control_Ub                       int64
Peptides Control_UbP                      int64
Peptides Control_WCL                      int64
Peptides Control_WCLP                     int64
Peptides Pynd_5FC_Ub                      int64
Peptides Pynd_5FC_UbP                     int64
Peptides Pynd_5FC_WCL                     int64
Peptides Pynd_5FC_WCLP                    int64
Peptides Pynd_AlkKO_Ub                    int64
Peptides Pynd_AlkKO_UbP                   int64
Peptides Pynd_AlkKO_WCL                 

/Users/student/anaconda/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (422,421) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,Protein IDs,Majority protein IDs,Peptide counts (all),Peptide counts (razor+unique),Peptide counts (unique),Fasta headers,Number of proteins,Peptides,Razor + unique peptides,Unique peptides,...,Peptide IDs,Peptide is razor,Mod. peptide IDs,Evidence IDs,MS/MS IDs,Best MS/MS,Oxidation (M) site IDs,Phospho (STY) site IDs,Oxidation (M) site positions,Phospho (STY) site positions
0,[CON__A2AB72],[CON__A2AB72],[1],[1],1,[>A2AB72 TREMBL:A2AB72 Tax_Id=10090 Gene_Symbo...,1,1,1,1,...,[16687],[True],[18612],[258339],[135445],[135445],[],"[0, 1, 7687, 9993]",[],"[305, 306, 307, 313]"
1,[CON__ENSEMBL:ENSBTAP00000023402],[CON__ENSEMBL:ENSBTAP00000023402],[1],[1],1,[>ENSEMBL:ENSBTAP00000023402 (Bos taurus) 46 k...,1,1,1,1,...,[24636],[True],[27875],"[385721, 385722, 385723]",[204113],[204113],[],"[2, 3, 4, 9994]",[],"[295, 297, 301, 306]"
2,[CON__P00761],[CON__P00761],[8],[8],7,[>P00761 SWISS-PROT:P00761|TRYP_PIG Trypsin - ...,1,8,8,7,...,"[8497, 8498, 9030, 11741, 11742, 13051, 19953,...","[True, True, True, True, True, True, True, True]","[9339, 9340, 9341, 9342, 9918, 13001, 13002, 1...","[132159, 132160, 132161, 132162, 132163, 13216...","[69949, 69950, 69951, 69952, 69953, 69954, 699...","[69957, 69979, 74399, 97924, 97927, 108901, 16...",[0],[],[94],[]
3,"[CON__P02533, CON__A2A4G1, CON__P08779, CON__P...","[CON__P02533, CON__A2A4G1, CON__P08779]","[3, 2, 2, 1, 1, 1, 1, 1, 1, 1]","[1, 0, 0, 1, 1, 0, 1, 0, 0, 0]",1;0;0;1;1;0;1;0;0;0,[>P02533 SWISS-PROT:P02533 Tax_Id=9606 Gene_Sy...,10,3,1,1,...,"[14059, 24240, 24241]","[True, False, False]","[15657, 27438, 27439, 27440]","[221849, 221850, 221851, 221852, 221853, 22185...","[117197, 200285, 200286, 200287, 200288, 20028...","[117197, 200288, 200290]",[1],[],[119],[]
4,[CON__P02662],[CON__P02662],[1],[1],1,[>P02662 SWISS-PROT:P02662 Alpha-S1-casein - B...,1,1,1,1,...,[25121],[True],[28412],"[394578, 394579, 394580, 394581, 394582, 39458...","[208746, 208747, 208748, 208749, 208750, 20875...",[208746],[],[5],[],[115]
5,[CON__P02663],[CON__P02663],[1],[1],1,[>P02663 SWISS-PROT:P02663 Alpha-S2-casein [Co...,1,1,1,1,...,[22607],[True],[25637],"[349807, 349808, 349809]",[184665],[184665],[2],"[6, 7688]",[141],"[143, 144]"
6,[CON__P02666],[CON__P02666],[2],[2],2,[>P02666 SWISS-PROT:P02666 Beta-casein - Bos t...,1,2,2,2,...,"[3996, 5453]","[True, True]","[4412, 5977]","[62185, 62186, 62187, 62188, 62189, 62190, 621...","[32906, 43265, 43266, 43267, 43268, 43269, 432...","[32906, 43270]",[],[7],[],[35]
7,[CON__P02769],[CON__P02769],[25],[25],21,[>P02769 SWISS-PROT:P02769 (Bos taurus) Bovine...,1,25,25,21,...,"[326, 1994, 2150, 2168, 3266, 3401, 4669, 5189...","[True, True, True, True, True, True, True, Tru...","[369, 2251, 2414, 2432, 3613, 3764, 5149, 5701...","[6087, 36083, 37979, 37980, 38315, 52390, 5239...","[3472, 19615, 20730, 20731, 20901, 28146, 2814...","[3472, 19615, 20730, 20901, 28146, 29030, 3724...",[],[],[],[]
8,"[CON__P04264, CON__Q9R0H5, CON__Q6NXH9, CON__Q...",[CON__P04264],"[27, 1, 1, 1, 1]","[27, 1, 1, 1, 1]",19;0;0;0;0,[>P04264 SWISS-PROT:P04264 Tax_Id=9606 Gene_Sy...,5,27,27,19,...,"[289, 290, 5264, 5265, 6052, 6098, 6703, 6704,...","[True, True, True, True, True, True, True, Tru...","[331, 332, 5778, 5779, 6616, 6667, 7340, 7341,...","[5489, 5490, 5491, 5492, 5493, 5494, 5495, 549...","[3184, 3185, 3186, 41967, 41968, 41969, 41970,...","[3185, 3186, 41972, 41978, 47752, 48000, 52590...",[],[8],[],[399]
9,"[CON__Q3KNV1, CON__P08729]","[CON__Q3KNV1, CON__P08729]","[1, 1]","[1, 1]",1;1,"[>Q3KNV1 TREMBL:Q3KNV1, Q96GE1 Tax_Id=9606 Gen...",2,1,1,1,...,[13036],[True],[14422],"[205598, 205599, 205600, 205601]",[108786],[108786],[],[9],[],[38]


In [203]:
def parse_phosphosites(f):
    df = pandas.read_table(f, header=0)

    # These are lists
    for m in [
            'Proteins',
            'Leading proteins',
            'Fasta headers'
        ]:
            df[m] = df[m].map(lambda s: [st.strip() for st in str(s).split(';')])
            
    # And these are lists of ints
    for m in [
            'Positions within proteins'
    ]:
        df[m] = df[m].map(lambda s: [int(st) for st in str(s).split(';')] if str(s) != 'nan' else [])

    # TODO Weird data structures
    
    return df



In [204]:
phospho = parse_phosphosites(phospho_peptide_file)
phospho = filter_other_groups(phospho)
print(phospho.dtypes)
phospho

Proteins                                           object
Positions within proteins                          object
Leading proteins                                   object
Protein                                            object
Fasta headers                                      object
Localization prob                                 float64
Score diff                                        float64
PEP                                               float64
Score                                             float64
Delta score                                       float64
Score for localization                            float64
Localization prob Apubscrawl_Cerulenin_Ub         float64
Score diff Apubscrawl_Cerulenin_Ub                float64
PEP Apubscrawl_Cerulenin_Ub                       float64
Score Apubscrawl_Cerulenin_Ub                     float64
Localization prob Apubscrawl_Cerulenin_UbP        float64
Score diff Apubscrawl_Cerulenin_UbP               float64
PEP Apubscrawl

/Users/student/anaconda/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (211,677,678) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,Proteins,Positions within proteins,Leading proteins,Protein,Fasta headers,Localization prob,Score diff,PEP,Score,Delta score,...,Best localization raw file,Best localization scan number,Best score evidence ID,Best score MS/MS ID,Best score raw file,Best score scan number,Best PEP evidence ID,Best PEP MS/MS ID,Best PEP raw file,Best PEP scan number
0,[CON__A2AB72],[306],[CON__A2AB72],CON__A2AB72,[>A2AB72 TREMBL:A2AB72 Tax_Id=10090 Gene_Symbo...,0.319430,0.000000,1.677750e-02,49.176,16.6280,...,QE20150928-62,26692,258339,135445,QE20150928-62,26692,258339,135445,QE20150928-62,26692
1,[CON__A2AB72],[307],[CON__A2AB72],CON__A2AB72,[>A2AB72 TREMBL:A2AB72 Tax_Id=10090 Gene_Symbo...,0.335129,0.000000,1.677750e-02,49.176,16.6280,...,QE20150928-62,26692,258339,135445,QE20150928-62,26692,258339,135445,QE20150928-62,26692
2,[CON__ENSEMBL:ENSBTAP00000023402],[295],[CON__ENSEMBL:ENSBTAP00000023402],CON__ENSEMBL:ENSBTAP00000023402,[>ENSEMBL:ENSBTAP00000023402 (Bos taurus) 46 k...,0.998267,26.943800,2.022300e-02,43.690,19.2100,...,QE20150928-57,4052,385721,204113,QE20150928-57,4052,385721,204113,QE20150928-57,4052
3,[CON__ENSEMBL:ENSBTAP00000023402],[297],[CON__ENSEMBL:ENSBTAP00000023402],CON__ENSEMBL:ENSBTAP00000023402,[>ENSEMBL:ENSBTAP00000023402 (Bos taurus) 46 k...,0.993682,21.325300,2.022300e-02,43.690,19.2100,...,QE20150928-57,4052,385721,204113,QE20150928-57,4052,385721,204113,QE20150928-57,4052
4,[CON__ENSEMBL:ENSBTAP00000023402],[301],[CON__ENSEMBL:ENSBTAP00000023402],CON__ENSEMBL:ENSBTAP00000023402,[>ENSEMBL:ENSBTAP00000023402 (Bos taurus) 46 k...,0.943054,11.776500,2.022300e-02,43.690,19.2100,...,QE20150928-57,4052,385721,204113,QE20150928-57,4052,385721,204113,QE20150928-57,4052
5,[CON__P02662],[115],[CON__P02662],CON__P02662,[>P02662 SWISS-PROT:P02662 Alpha-S1-casein - B...,1.000000,118.081000,3.517030e-14,152.630,120.4800,...,QE20150928-101,14221,394578,208746,QE20150928-55,16909,394578,208746,QE20150928-55,16909
6,[CON__P02663],[143],[CON__P02663],CON__P02663,[>P02663 SWISS-PROT:P02663 Alpha-S2-casein [Co...,0.499898,0.000000,1.012980e-02,77.240,48.3360,...,QE20150928-55,13198,349807,184665,QE20150928-55,13198,349807,184665,QE20150928-55,13198
7,[CON__P02666],[35],[CON__P02666],CON__P02666,[>P02666 SWISS-PROT:P02666 Beta-casein - Bos t...,1.000000,110.948000,4.784720e-39,215.430,170.7900,...,QE20150928-92,12555,83043,43270,QE20150928-92,12555,83043,43270,QE20150928-92,12555
8,"[CON__P04264, CON__P35908]","[399, 403]","[CON__P04264, CON__P35908]",CON__P04264,[>P04264 SWISS-PROT:P04264 Tax_Id=9606 Gene_Sy...,1.000000,61.343800,7.426560e-03,61.344,16.1310,...,QE20150928-94,9571,126099,66769,QE20150928-94,9571,126099,66769,QE20150928-94,9571
9,"[CON__Q3KNV1, CON__P08729]","[38, 38]",[CON__Q3KNV1],CON__Q3KNV1,"[>Q3KNV1 TREMBL:Q3KNV1, Q96GE1 Tax_Id=9606 Gen...",0.585170,4.825630,3.766950e-03,47.350,22.4460,...,QE20150928-45,16948,205598,108786,QE20150928-45,16948,205598,108786,QE20150928-45,16948


In [ ]:
merged = phospho.merge(proteins_raw, how='inner', left_on='Protein Group IDs', right_on='id')
merged